# Air netCDF File for All Variables (mean, clim, anom,std)

In [1]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import datetime
from dateutil.relativedelta import relativedelta

In [36]:
os.chdir('d:/NCEP_R2')
ds_air= xr.open_dataset('air.2m.mon.mean.nc')

In [41]:
# ds_air #check

In [39]:
## unit change
ds_air['air'] = ds_air.air-273.15

## change dim
ds_air = ds_air.squeeze('level')
ds_air = ds_air.drop('level', dim=None)

ds_air # check

<xarray.Dataset>
Dimensions:  (time: 545, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2024-05-01
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    air      (time, lat, lon) float32 -33.48 -33.51 -33.53 ... -45.3 -45.48

## cut new clim

In [42]:
# new_clim_ds
new_clim = ds_air.sel(time=slice('1991-01-01','2020-12-01'))
new_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    air      (time, lat, lon) float32 -24.83 -24.6 -24.53 ... -24.54 -24.63

In [44]:
### add clim variable / len  30
clim_sel = new_clim['air'].groupby('time.month').mean('time')
new_clim['new_clim'] = clim_sel

### anom sel
# case 1 
anom_lst = []

for i in range(1, 13):
    each_mon = new_clim['air'].sel(time=new_clim['time.month'] == i)  # month
    clim_mon = new_clim.new_clim.sel(month=i)  # clim
    anom_sel = each_mon - clim_mon  # anom
    
    anom_lst.append(anom_sel)

anom_cb = xr.concat(anom_lst, dim='time')
new_clim['new_anom'] = anom_cb

### std sel
std_sel=new_clim.air.groupby('time.month').std(dim='time',ddof=1)

std_cb = xr.concat(std_sel, dim='month')  
new_clim['new_std'] = std_cb

new_clim.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
new_clim.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
new_clim.air.attrs ={'unit' : 'deg C', 'long_name':'NCEP Monthly Air Temperature at 2m'}
new_clim.new_clim.attrs ={'units':'deg C','long_name':'Average Long Term Mean Monthly Rate of Air Temperature at 2m'}
new_clim.new_anom.attrs ={'units':'deg C','long_name':'The Monthly Anomaly Rate of Forecasted Air Temperature at 2m'} 
new_clim.new_std.attrs = {'units':'deg C','long_name':'Standard Deviation Rate of Forecasted Air Temperature at 2m'}
# new_clim

c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [45]:
new_clim

<xarray.Dataset>
Dimensions:   (time: 360, lat: 94, lon: 192, month: 12)
Coordinates:
  * time      (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon       (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat       (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * month     (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    air       (time, lat, lon) float32 -24.83 -24.6 -24.53 ... -24.54 -24.63
    new_clim  (month, lat, lon) float32 -26.75 -26.6 -26.63 ... -26.9 -27.0
    new_anom  (time, lat, lon) float32 1.916 1.996 2.099 ... 2.349 2.357 2.367
    new_std   (month, lat, lon) float32 4.267 4.253 4.297 ... 2.2 2.21 2.219

In [46]:
os.getcwd()
# new_clim.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/NCEP_air_ncfile_all_variables(mean,clim,anom,std)_new')

## cut old clim

In [47]:
old_clim = ds_air.sel(time=slice('1981-01-01','2010-12-01'))
old_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 94, lon: 192)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Data variables:
    air      (time, lat, lon) float32 -30.21 -30.08 -30.82 ... -27.47 -27.54

In [48]:
### add clim variable / len  30
clim_sel = old_clim['air'].groupby('time.month').mean('time')
old_clim['old_clim'] = clim_sel

### anom sel
# case 1 
anom_lst = []

for i in range(1, 13):
    each_mon = old_clim['air'].sel(time=old_clim['time.month'] == i)  # month
    clim_mon = old_clim.old_clim.sel(month=i)  # clim
    anom_sel = each_mon - clim_mon  # anom
    
    anom_lst.append(anom_sel)

anom_cb = xr.concat(anom_lst, dim='time')
old_clim['old_anom'] = anom_cb

### std sel
std_sel=old_clim.air.groupby('time.month').std(dim='time',ddof=1)

std_cb = xr.concat(std_sel, dim='month')  
old_clim['old_std'] = std_cb

old_clim.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
old_clim.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
old_clim.air.attrs ={'unit' : 'deg C', 'long_name':'NCEP Monthly Air Temperature at 2m'}
old_clim.old_clim.attrs ={'units':'deg C','long_name':'Average Long Term Mean Monthly Rate of Air Temperature at 2m'}
old_clim.old_anom.attrs ={'units':'deg C','long_name':'The Monthly Anomaly Rate of Forecasted Air Temperature at 2m'} 
old_clim.old_std.attrs = {'units':'deg C','long_name':'Standard Deviation Rate of Forecasted Air Temperature at 2m'}
# old_clim

c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [49]:
old_clim

<xarray.Dataset>
Dimensions:   (time: 360, lat: 94, lon: 192, month: 12)
Coordinates:
  * time      (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon       (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat       (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * month     (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    air       (time, lat, lon) float32 -30.21 -30.08 -30.82 ... -27.47 -27.54
    old_clim  (month, lat, lon) float32 -27.9 -27.82 -27.84 ... -27.0 -27.09
    old_anom  (time, lat, lon) float32 -2.309 -2.262 -2.979 ... -0.474 -0.4473
    old_std   (month, lat, lon) float32 4.412 4.375 4.396 ... 2.214 2.226 2.235

In [50]:
os.getcwd()
# old_clim.to_netcdf('8.NCEP_ncfile(mean,clim,anom,std)/NCEP_air_ncfile_all_variables(mean,clim,anom,std)_old')